# Python для сбора и обработки данных

Автор: Валентин Бирюков

# Краудсорсинг, ML и продакшон

Вспомним, что мы работали в проекте "Крутые вещи и не только", и уже успели поковыряться для нашего мелкого стартапа с аналитикой.
Мы подготовили всю продукцию, провели все сравнительные анализы, бэкендеры написали бэкенд, фронтендеры написали фронтенд... В общем запуск через пару дней, "все идет по плану".

Почти. Тут наши, допустим, продуктологи вспоминают что вообще-то мы предложили "аааабсолютно" новую возможность - обсуждать товар прям рядом с карточкой. Не только озывы после покупки, которые может оставить уже не так уж много людей - но прям в режиме живого времени комментировать классный это товар или нет (еще бы, ведь мы хотим поддерживать самую свежую базу товаров).

Но люди чуть-чуть не вписались в нашу модель. Они ведь могут оставить непойми что! Хоть мат, хоть рекламу других сервисов. Надо что-то придумывать.

Окей, базовый мат вырежем регулярками, ссылки запретим публиковать, но за всем не усмотришь, а русский язык велик и могуч! И вот вся надежда на нас, аналитиков, что мы то уж придумаем, как разобраться с такими данными.

Варианты наши одновременно обширны и скудны:
- помочь писать регулярки?
- обучить модель? но где найти разметку?
- штат модерации? нанимать людей, писать новые доступы в админку - сорвем все запуски.

И, наслушавшись лекций мы решаем пребегнуть к краудсорсингу.

В рамках данного Задания мы попробуем построить кусочки для "автоматического" (масштабируемого, дешевого) анализа текстовых комментариев. В качестве источника таковых комментариев возьмем Двач (анонимную страницу), и попробуем отмодерировать коментарии на ней

# Начальная заготовка

In [10]:
import urllib
import urllib.request
from bs4 import BeautifulSoup
import re

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

headers={'User-Agent':user_agent,} 

def make_soup(url):
    print(url)
    request= urllib.request.Request(url, None,headers)
    thepage = urllib.request.urlopen(request, )
    soupdata = BeautifulSoup(thepage, "html.parser")
    return soupdata

# Научимся "обходить" двач

В рамках данной работы делать этого мы, конечно же не будем, но бдительный читатель и сам прикинет как это сделать - оно похожее на обход разных категорий в сборе данных с других площадок.

# Соберем данные из одной страницы для разметки

In [11]:
page = 'https://2ch.hk/pr/res/1647485.html'

In [38]:
def clean(article):
    return article.get_text().strip()
    

soup = make_soup(page)

text = list(map(clean, soup.findAll('article', {'class': 'post__message'})))

https://2ch.hk/pr/res/1647485.html


In [39]:
text

['https://github.com/jwasham/coding-interview-university - теорияhttps://leetcode.com - задачки',
 'Вкатыши обезумели, сумеем ли мы совладать с ними? Кто возьмёт на высоконагруженный бэк человека без опыта, но с дипломом вайти-курсов? Хотелось бы почитать отзывы через полтора года когда Go здохнет, как червь в анусе шелудивого пса',
 '>>1806968>когда Go здохнетТолько вот почему-то высоконагруженный бэк переезжает со Scala/Java/PHP/Ruby как раз на Go и показывает превосходные результаты...',
 '>>1806970Это все веяние моды, порождаемое google',
 '>>1806970Гугл не умеет в языки. Он их уже очень много высрал, последи за их судьбою.',
 '>>1806974>Гугл не умеет в языкиНу, началось! Возьми JetBrains и их Kotlin, который теперь является основным языком разработки под Android, может в мультиплатформу и многие проекты с Java переползают на Kotlin либо стартуют проекты на нем. С выходом Kotlin, Scala стала не нужна, между прочим и теперь она доживает последние деньки до выхода Dotty (Scala 3).',


# Подключимся к толоке

проекты/инструкции и все такое подготовим один раз в интерфейсе - как никак один раз может проще и руками.

In [41]:
import requests

TOKEN = 'TOKEN'
header = {
    'Authorization': 'OAuth '+TOKEN,
    'Content-Type': 'application/JSON'
}

base_url = 'https://toloka.yandex.ru'

'{"request_id":"eecaffa8c9c83e403b31b0d83a490b25","payload":{"":{"code":"ARRAY_SIZE_LESS_THAN_MIN","message":"Array size must be greater or equal to 1","params":[1]}},"code":"VALIDATION_ERROR","message":"Validation failed"}'

In [64]:
# клонируем базовый пул
import time

def clone_pool_kostyl(pool_id):
    response = requests.post(
        base_url+'/api/v1/pools/{}/clone'.format(pool_id),
        headers=header
    )
    operation_id = response.json()['id']
    response = requests.get(
        base_url+'/api/v1/operations/{}'.format(operation_id), headers=header
    )
    status = response.json()['status']
    while status != 'SUCCESS':
        time.sleep(1)
        response = requests.get(
            base_url+'/api/v1/operations/{}'.format(operation_id), headers=header
        )
        status = response.json()['status']
    return response.json()['details']['pool_id']

In [73]:
pool_to_upload = clone_pool_kostyl('16548328')

In [74]:
tasks = [
    {
        'pool_id': pool_to_upload,
        'input_values': {'comment': comment}
    } for comment in text
]
tasks[:2]

[{'pool_id': '16549997',
  'input_values': {'comment': 'https://github.com/jwasham/coding-interview-university - теорияhttps://leetcode.com - задачки'}},
 {'pool_id': '16549997',
  'input_values': {'comment': 'Вкатыши обезумели, сумеем ли мы совладать с ними? Кто возьмёт на высоконагруженный бэк человека без опыта, но с дипломом вайти-курсов? Хотелось бы почитать отзывы через полтора года когда Go здохнет, как червь в анусе шелудивого пса'}}]

# Загрузим все данные

In [ ]:
#r = requests.post(base_url+'/api/v1/tasks?allow_defaults=true', headers=header, json=tasks)
#r.text

# Получим результат разметки

In [93]:
assignments = requests.get(
    base_url+'/api/v1/assignments?pool_id={}'.format(pool_to_upload),
    headers=header
)

In [94]:
a = [1,2,3]
b = [4,5,6]

In [103]:
assignments.json()['items'][0]

{'id': '0000fc886d--5f625876bebe374a2dc7f647',
 'task_suite_id': '0000fc886d--5f625876bebe374a2dc7f645',
 'pool_id': '16549997',
 'user_id': '707501a1ab6b15bc1f9d45a9132b062a',
 'status': 'ACCEPTED',
 'reward': 0.01,
 'tasks': [{'id': '0000fc886d--5f625735dc9921307992b748',
   'input_values': {'comment': '>>1807207жить. мoжет пoвезёт найти рабoту, а мoжет и нет. всё равнo делать бoльше нечегo. я ещё сo шкoлы тестерoм и паяльникoм платы тыкал, в веб вкатываться этo вooбще не мoё меня прям блевать тянет oт всех этих сайтoв. а вoт с\\с++, oперациoнные системы, ассемблеры, драйвера, этo тo чтo мне нравится.'},
   'pool_id': '16549997',
   'overlap': 3,
   'infinite_overlap': False,
   'reserved_for': [],
   'unavailable_for': [],
   'created': '2020-09-16T18:19:33.579',
   'remaining_overlap': 0},
  {'id': '0000fc886d--5f625743dc9921307992bb14',
   'input_values': {'comment': 'Как же меня заебала уже эта ленивая догрузка сайтов и я постоянно промахиваюсь мимо ссылок, в основном логина, и п

In [104]:
results = []
task_suits = assignments.json()['items']
for suit in task_suits:
    if suit['status'] != 'ACCEPTED':
        continue
    input_values = [item['input_values'] for item in suit['tasks']]
    output_values = [item['output_values'] for item in suit['solutions']]
    for i in range(len(input_values)):
        input_values[i].update(output_values[i])
    results.extend(input_values)
results

[{'comment': '>>1807207жить. мoжет пoвезёт найти рабoту, а мoжет и нет. всё равнo делать бoльше нечегo. я ещё сo шкoлы тестерoм и паяльникoм платы тыкал, в веб вкатываться этo вooбще не мoё меня прям блевать тянет oт всех этих сайтoв. а вoт с\\с++, oперациoнные системы, ассемблеры, драйвера, этo тo чтo мне нравится.',
  'advertising': False,
  'nonsense': False,
  'insult': False,
  'law_violation': False,
  'profanity': False,
  'quality': 'OK'},
 {'comment': 'Как же меня заебала уже эта ленивая догрузка сайтов и я постоянно промахиваюсь мимо ссылок, в основном логина, и попадаю в банеры. Две самые сука дебильные технологии это ленивая подгрузка и бесконечная подгружаемая лента.',
  'advertising': False,
  'nonsense': False,
  'insult': False,
  'law_violation': False,
  'profanity': False,
  'quality': 'OK'},
 {'comment': ">>1808236I haven't commercial products yet.О, ви из Англии?",
  'advertising': False,
  'nonsense': False,
  'insult': False,
  'law_violation': False,
  'profanit

# Проведем аггрегацию результатов